<a href="https://colab.research.google.com/github/ymuto0302/base1_2021/blob/main/opendata3_GoogleCommunityMobilityReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オープンデータの活用(5) : Google's COVID-19 Community Mobility Reports を用いた行動解析

これまで日本国内のデータを利用してきたが，今回は新型コロナウイルス関連で世界中をカバーする "Google's COVID-19 Community Mobility Reports" を利用する。

Community Mobility Reports とは，(天下の)Google がロケーション履歴を元に集計したレポートであり，「食料品店，乗換駅などの特定の場所を訪れた人数の変化率」をオープンデータ化している。


---

## Google Community Mobility Reports のデータを見てみよう

### Google Community Mobility Reports からのデータ入手
下記の COVID-19: コミュニティ モビリティ レポートからデータを入手する。

https://www.google.com/covid19/mobility/

ページ内の中段にある「全世界のCSVをダウンロード」をクリックし，ファイル "Global_Mobility_Report.csv" を Google Drive に置いておく。


### Google Drive のマウント & plotly express のインストール
plotly express は前回も用いた「可視化ライブラリ」である。

In [1]:
# Google Drive のマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 可視化の準備：PlotlyExpress のインストール
!pip install plotly_express

### Google's Community Mobility Reports のデータを眺めてみる
最初に Google Community Mobility Reports の全容を把握する。
下記のとおり，2021年4月22日時点でのデータ数は 5,039,384件であり，13列から構成される。

#### 各列の意味
"country_region_code" が国コードを表し，日本は "JP" である。
また，"sub_region_1" には都道府県名や州名が入る。

データフレームの右の方を眺めるとデータ観測日である date に続き，特定の場所（食料品店、公園など）を訪れた人数の変化率が示されている。「人数の変化率」を観察したポイントは次の６項目からなる。

- retail_and_recreation_percent_change_from_baseline # 小売店と娯楽施設(対象: レストラン、カフェ、ショッピング センター、テーマパーク、博物館、図書館、映画館など)
- grocery_and_pharmacy_percent_change_from_baseline # 食料品店と薬局(対象: 食料品店、食品問屋、青果市場、高級食料品店、ドラッグストア、薬局など)
- parks_percent_change_from_baseline # 公園(対象: 地域の公園、国立公園、公共のビーチ、マリーナ、ドッグパーク、広場、庭園など)
- transit_stations_percent_change_from_baselin # 公共交通期間(対象: 公共交通機関の拠点（例: 地下鉄、バス、電車の駅）など)
- workplaces_percent_change_from_baseline # 職場(対象: 職場)
- residential_percent_change_from_baseline # 住宅(対象: 住居)

また，「人数の変化」はパーセント表示であり，（Google Community Mobility Reports によると）その曜日別基準値は「2020年1月3日〜2020年2月6日の５週間の曜日別中央値」を用いている。

#### データ読み込み

In [3]:
import pandas as pd

# Google's COVID-19 Community Mobility Reports (COVID-19 コミュニティ モビリティ レポート) の読み込み
df = pd.read_csv("/content/drive/My Drive/BASE/OpenData/Global_Mobility_Report.csv", encoding="utf-8")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 読み込んだデータフレームの表示
df

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5039379,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,ChIJRcIZ3-FJNBkRRsj55IcLpfU,2021-04-12,NaN,NaN,NaN,NaN,7.0,NaN
5039380,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,ChIJRcIZ3-FJNBkRRsj55IcLpfU,2021-04-13,NaN,NaN,NaN,NaN,14.0,NaN
5039381,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,ChIJRcIZ3-FJNBkRRsj55IcLpfU,2021-04-14,NaN,NaN,NaN,NaN,9.0,NaN
5039382,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,ChIJRcIZ3-FJNBkRRsj55IcLpfU,2021-04-15,NaN,NaN,NaN,NaN,13.0,NaN


#### 可視化へ向けた前処理
1. 解析上，不要と考えられる列 "metro_area", "iso_3166_2_code", "census_fips_code", "place_id" を削除する。ここでは drop() メソッドを用いる。
1. 観察ポイントにおける変化率は "retail_and_recreation_percent_change_from_baseline" という，長い英語表記となっている。これを端的な日本語に置き換える。ここでは rename() メソッドを用いる。

In [5]:
# 不要と考えられる列をカット
df = df.drop(['metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id'], axis=1)

# 列名が長いため，短縮かつ日本語化する
# (メモ) rename の書式 : columns={変更前A:変更後A, 変更前B:変更後B}
df = df.rename(columns={'retail_and_recreation_percent_change_from_baseline':'小売・娯楽',
                        'grocery_and_pharmacy_percent_change_from_baseline': '食料品店・薬局',
                        'parks_percent_change_from_baseline': '公園',
                        'transit_stations_percent_change_from_baseline': '乗換駅',
                        'workplaces_percent_change_from_baseline': '職場',
                        'residential_percent_change_from_baseline': '住宅'})

In [6]:
# データフレームの冒頭部分の表示
df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,小売・娯楽,食料品店・薬局,公園,乗換駅,職場,住宅
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


#### (参考) どれだけの国・地域をカバーしているか？
以下のとおり 135国・地域をカバーしたデータである。

In [7]:
len(set(list(df['country_region'])))


135

#### データの中身を理解するため，東京のデータのみ取り出す
全世界のデータから**東京のデータのみを取り出す**。具体的には "country_region_code" が "JP"(日本)，かつ "sub_region_1" が "Tokyo"(東京)に該当する行のみを取り出す。

データを絞り込んだ結果，20496件のデータが得られた。

In [8]:
# 東京のデータのみに絞り込む
df_tokyo = df.copy() # 大元のデータフレーム df を壊したくないから，コピーをとる
condition = (df['country_region_code'] == 'JP') & (df['sub_region_1'] == 'Tokyo') # 絞り込むための条件
df_tokyo = df_tokyo[condition] # 絞り込み

In [9]:
# データフレームの冒頭部分の表示
df_tokyo.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,小売・娯楽,食料品店・薬局,公園,乗換駅,職場,住宅
2517324,JP,Japan,Tokyo,NaN,2020-02-15,-2.0,5.0,9.0,1.0,4.0,1.0
2517325,JP,Japan,Tokyo,NaN,2020-02-16,-13.0,-9.0,-37.0,-9.0,-1.0,3.0
2517326,JP,Japan,Tokyo,NaN,2020-02-17,-4.0,3.0,-2.0,-1.0,1.0,0.0
2517327,JP,Japan,Tokyo,NaN,2020-02-18,-2.0,3.0,3.0,-3.0,0.0,1.0
2517328,JP,Japan,Tokyo,NaN,2020-02-19,-4.0,2.0,8.0,-5.0,-1.0,1.0


#### plotly express を用いて，東京の移動状況を可視化

In [12]:
# 東京での人の移動状況を折れ線グラフで表示
import plotly_express as px

df_melt = df_tokyo.melt(id_vars='date', value_vars=["小売・娯楽", "食料品店・薬局", "公園", "乗換駅", "職場", "住宅"])
fig = px.line(df_melt, x="date", y="value", color='variable', title="Google Mobility Report")
fig.show()

# (メモ) Anaconda 3 での記述
#fig = px.line(df_tokyo, x="date", y=["小売・娯楽", "食料品店・薬局", "公園", "乗換駅", "職場", "住宅"],
#              title="Google Mobility Report")

#### 曜日効果が含まれ分かりにくいため，７日移動平均をとる
上記の結果は「週末に職場への人の移動が減少する」また「公園は週末に人出が増える」等の曜日効果が含まれ，細かく振動している。

そこで，トレンドを把握するため，７日移動平均をとる。

移動平均(moving average)は pandas の機能に含まれており，rolling().mean() を用いる。

In [13]:
window = 7 # ウィンドウ幅

# 移動平均を求める
rolling_mean = df_tokyo[['小売・娯楽', '食料品店・薬局', '公園', '乗換駅', '職場', '住宅']].rolling(window).mean()

# date 列と結合する
rolling_mean = pd.concat([df_tokyo['date'], rolling_mean], axis=1)

# 冒頭の６日間は値が存在しないため，これをカットする（windows-1 行目以降を取り出す)
rolling_mean = rolling_mean[window-1:] # 頭の window 分を削除

In [14]:
# (参考) ７日移動平均の確認。ここでスタート日が 2020年2月15日から 2020年2月21日へ変更されている点に注意する。
rolling_mean.head()

,date,小売・娯楽,食料品店・薬局,公園,乗換駅,職場,住宅
2517330,2020-02-21,-4.714286,1.142857,-2.857143,-3.857143,0.000000,1.285714
2517331,2020-02-22,-5.714286,0.428571,-5.142857,-4.714286,-0.714286,1.428571
2517332,2020-02-23,-4.285714,2.142857,2.142857,-4.142857,-0.714286,1.000000
2517333,2020-02-24,-4.428571,1.571429,6.857143,-9.714286,-10.285714,3.714286
2517334,2020-02-25,-5.000000,1.857143,5.714286,-10.000000,-10.428571,3.714286


In [ ]:
#### 改めて東京の移動状況を可視化

In [17]:
import plotly_express as px

df_melt = rolling_mean.melt(id_vars='date', value_vars=["小売・娯楽", "食料品店・薬局", "公園", "乗換駅", "職場", "住宅"])
fig = px.line(df_melt, x="date", y="value", color='variable', title="Google Mobility Report (Tokyo)")
fig.show()

# (メモ) Anaconda 3 での記述
#fig = px.line(rolling_mean, x='date', y=['小売・娯楽', '食料品店・薬局', '公園', '乗換駅', '職場', '住宅'],
#              title="Google Mobility Report (Tokyo)")

上記のグラフより見えてくることは以下のとおり：
- 第１波の頃，職場における人の往来は40%以上，減少した。一方，住宅への滞在は 20%超，増加している。（注意：-60% に至ったのは GW のためである）
- １回目の緊急事態宣言(2020年4月7日)以降はしばらく職場への人の往来は減少傾向にあった。
- 一方，２回目の緊急時事態宣言(2021年1月7日)以降は，職場の減少は見られないが，公園に出掛ける（つまり，外出を目的とする）人出は減少した。
- 食料品店・薬局は，（やや減少しているものの）概ね変化は小さい。これは生活必需品の購入が要因であろう。
- 小売・娯楽施設は，期間を通じて人の往来が減っている。

以上より，新型コロナ禍によって日本社会が大きな影響を受けいてることが明らかである。

#### (参考) 同じデータ処理を山口県でやってみると・・・
祝日効果を除くと，期間を通じて変化の小さいことが分かる。

In [18]:
# 日本のデータのみ絞り込む (# データ数 = 20496件)
df_yamaguchi = df.copy() # 大元のデータフレーム df を壊したくないから，コピーをとる
condition = (df['country_region_code'] == 'JP') & (df['sub_region_1'] == 'Yamaguchi') # 絞り込むための条件
df_yamaguchi = df_yamaguchi[condition] # 絞り込み

# 移動平均を求める
window = 7 # ウィンドウ幅
rolling_mean = df_yamaguchi[['小売・娯楽', '食料品店・薬局', '公園', '乗換駅', '職場', '住宅']].rolling(window).mean()
rolling_mean = pd.concat([df_yamaguchi['date'], rolling_mean], axis=1)
rolling_mean = rolling_mean[window-1:] # 頭の window 分を削除

# 可視化
import plotly_express as px
df_melt = rolling_mean.melt(id_vars='date', value_vars=["小売・娯楽", "食料品店・薬局", "公園", "乗換駅", "職場", "住宅"])
fig = px.line(df_melt, x="date", y="value", color='variable', title="Google Mobility Report (Tokyo)")
fig.show()

---

# ここから本気で解析！

---

職場への移動，休日の外出の抑制状況を見るには「乗換駅」での人出を評価するのが適切であろう。

## 日本の都道府県を対象とした解析

### 日本のデータのみを取り出す
日本のデータのみを取り出すには df['country_region_code'] == 'JP' という条件を与えればよい。

In [19]:
df_japan = df.copy() # 大元のデータフレーム df を壊したくないから，コピーをとる
condition = (df['country_region_code'] == 'JP') # 絞り込むための条件
df_japan = df_japan[condition] # 絞り込み

### 特定の都道府県のみを取り出す
ピボット(pivot)を用いてデータを並べ替える。

日本のデータのみを含むデータフレーム df_japan に対して，sub_region_1 の値を指定し，特定の都道府県のデータを取り出す。

#### ピボットを用いた並び替え

In [27]:
# 列を sub_region_1 (都道府県名)，表中の値として "乗換駅" を指定し，ピポット
# その後，"date" (年月日)列と結合
japan = pd.concat([df_japan['date'], df_japan.pivot(columns='sub_region_1', values='乗換駅')], axis=1)
japan.head()

,date,NaN,Aichi,Akita,Aomori,Chiba,Ehime,Fukui,Fukuoka,Fukushima,Gifu,Gunma,Hiroshima,Hokkaido,Hyogo,Ibaraki,Ishikawa,Iwate,Kagawa,Kagoshima,Kanagawa,Kochi,Kumamoto,Kyoto,Mie,Miyagi,Miyazaki,Nagano,Nagasaki,Nara,Niigata,Oita,Okayama,Okinawa,Osaka,Saga,Saitama,Shiga,Shimane,Shizuoka,Tochigi,Tokushima,Tokyo,Tottori,Toyama,Wakayama,Yamagata,Yamaguchi,Yamanashi
2499817,2020-02-15,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2499818,2020-02-16,-10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2499819,2020-02-17,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2499820,2020-02-18,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2499821,2020-02-19,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 分析対象地域を山口県，東京都，大阪府に絞る

In [28]:
# 分析対象地域の絞り込み
japan = japan[['date', 'Yamaguchi', 'Tokyo', 'Osaka']] # 特定の都道府県に絞り込む
japan = japan.fillna(0) # 値が NaN の箇所をゼロで埋める
japan = japan.groupby('date').sum() # グルーピングして集計

# 移動平均を求める
window = 7 # ウィンドウ幅
rolling_mean = japan[['Yamaguchi', 'Tokyo', 'Osaka']].rolling(window).mean()
# rolling_mean = pd.concat([japan['date'], rolling_mean], axis=1)
rolling_mean = rolling_mean[window-1:] # 頭の window 分を削除

In [36]:
rolling_mean.reset_index()

,date,Yamaguchi,Tokyo,Osaka
0,2020-02-21,0.714286,-3.857143,-3.714286
1,2020-02-22,1.714286,-4.714286,-4.857143
2,2020-02-23,6.142857,-4.142857,-4.000000
3,2020-02-24,8.428571,-9.714286,-8.714286
4,2020-02-25,9.142857,-10.000000,-8.571429
...,...,...,...,...
416,2021-04-12,-12.142857,-27.714286,-26.571429
417,2021-04-13,-13.000000,-27.857143,-26.857143
418,2021-04-14,-12.714286,-28.142857,-27.285714
419,2021-04-15,-12.000000,-28.000000,-27.285714


#### 可視化

In [38]:
import plotly_express as px
rolling_mean =  rolling_mean.reset_index() # 'date' をインデック列から外す
df_melt = rolling_mean.melt(id_vars="date", value_vars=["Yamaguchi", "Tokyo", "Osaka"])
fig = px.line(df_melt, x="date", y="value", color='variable', title="Google Mobility Report (Yamaguchi / Tokyo / Osaka)")
fig.show()

以上より分かることは以下のとおり：

- 東京都は大阪府に比べ，「乗換駅」における人出の減少率が大きい。
- 山口県は都市部に比べ，人出の減少率が小さい。

---

# 課題：世界の都市で「乗換駅への人出の変動率」を比較する
Google 社は COVID-19 Community Mobility Reports を用いた分析にて留意すべき点として，以下を挙げている。

1. 位置情報の精度や情報取得状況は地域によって異なるため国家間の比較への利用は推奨されない
1. 同一曜日との比較値であるため祝日には外れ値を取りやすい
1. 通常時でも他の場所と比べて人々の滞在時間が長い「住居」ではモビリティの増加率が大きくなりにくい

「国家間の比較への利用は推奨されない」とのことだが，通信環境が整備された大都市間の比較ならば問題ないだろう。

そこで，日本，イギリス，フランス，ドイツ，ニュージーランドそれぞれの**首都**を対象として，「乗換駅」における人出の変動率を比較する。

- 東京 (日本)
- ロンドン (イギリス)
- パリ (フランス)
- ベルリン (ドイツ)
- ウェリントン (ニュージーランド)

## 参考：都市の指定
それぞれの首都に対応するデータを抽出するには，以下の指定を行う。

- Tokyo / Berlin / Wellignton は "sub_region_1" まで指定すればよい。
- London / Pari は "sub_region_2" まで指定する必要がある。

|city|country_region_code|sub_region_1|sub_region_2|
|:-|:-|:-|:-|
|**Tokyo**|'JP'|'Tokyo'||
|**London**|'GB'|'Greater London'|'City of London'|
|**Paris**|'FR'|'Île-de-France'|'Paris'|
|**Berlin**|'DE'|'Berlin'||
|**Wellington**|'NZ'|'Wellington'||

## 参考：イギリスとフランスの地域区分はややこしい
- 'Greater London' :
グレーター・ロンドン（英: Greater London）は、イギリスおよびイングランドの首都ロンドンの行政区画を形成するリージョンの1つであり、地方長官職を任官する目的で設置されるカウンティの1つでもある。
- 'Île-de-France' :
パリ（仏: Paris[1]、巴里）は、フランスの首都。イル＝ド＝フランス地域圏の首府。
